In [1]:
import cv2
import numpy as np

In [2]:
apple = cv2.imread('summer.jpg')
apple = cv2.resize(apple, (512,512))
orange = cv2.imread('winter.jpg')
orange = cv2.resize(orange, (512, 512))
print(apple.shape)
print(orange.shape)
apple_orange = np.hstack((apple[:, :304], orange[:, 304:]))

(512, 512, 3)
(512, 512, 3)


In [3]:
#generate Gaussion pyramid for apple (7 layer)
apple_copy = apple.copy()
gp_apple = [apple_copy]
for i in range(6):
    apple_copy = cv2.pyrDown(apple_copy)
    gp_apple.append(apple_copy)
    print(gp_apple[i].shape)

(512, 512, 3)
(256, 256, 3)
(128, 128, 3)
(64, 64, 3)
(32, 32, 3)
(16, 16, 3)


In [4]:
#generate Gaussion pyramid for orange (7 layer)
orange_copy = orange.copy()
gp_orange = [orange_copy]
for i in range(6):
    orange_copy = cv2.pyrDown(orange_copy)
    gp_orange.append(orange_copy)

In [5]:
apple_copy = gp_apple[5]
lp_apple = [apple_copy]
for i in range(5,0,-1):
    gaussian_expanded = cv2.pyrUp(gp_apple[i])
    print(gp_apple[i-1].shape)
    print(gaussian_expanded.shape)
    laplacian =cv2.subtract(gp_apple[i-1], gaussian_expanded)
    lp_apple.append(laplacian)

(32, 32, 3)
(32, 32, 3)
(64, 64, 3)
(64, 64, 3)
(128, 128, 3)
(128, 128, 3)
(256, 256, 3)
(256, 256, 3)
(512, 512, 3)
(512, 512, 3)


In [6]:
orange_copy = gp_orange[5]
lp_orange = [orange_copy]
for i in range(5,0,-1):
    gaussian_expanded = cv2.pyrUp(gp_orange[i])
    laplacian =cv2.subtract(gp_orange[i-1], gaussian_expanded)
    lp_orange.append(laplacian)

In [7]:
#add apple and orange
apple_orange_pyramid = []
n = 0
for apple_lap, orange_lap in zip (lp_apple, lp_orange):
    n += 1
    rows, cols, ch = apple_lap.shape
    laplacian = np.hstack((apple_lap[:, 0: int(cols/2)], orange_lap[:, int(cols/2):]))
    apple_orange_pyramid.append(laplacian)

In [8]:
#reconstruct
apple_orange_reconstruct = apple_orange_pyramid[0]
for i in range(1, 6):
    apple_orange_reconstruct = cv2.pyrUp(apple_orange_reconstruct)
    apple_orange_reconstruct = cv2.add(apple_orange_pyramid[i], apple_orange_reconstruct)
    
cv2.imshow("apple", apple)
cv2.imshow("orange", orange)
cv2.imshow("apple_orange", apple_orange)
cv2.imshow("apple_orange_reconstruct", apple_orange_reconstruct)
cv2.waitKey(0)
cv2.destroyAllWindows()